In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        assert os.path.exists(X_path), f"X_path {X_path} does not exist."
        assert os.path.exists(y_path), f"y_path {y_path} does not exist."
        
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1612, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1612,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  # Use appropriate preprocessing
        y_batch = np.eye(2)[self.y[batch_indices]]  # Ensure correct one-hot encoding
        return X_batch, y_batch

# Updated Callbacks
class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None:
            if current_value < self.threshold:
                self.wait += 1
                if self.wait >= self.patience:
                    print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                    self.model.stop_training = True
            else:
                self.wait = 0

# Updated Model Building Function
def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False

    sequence_input = Input(shape=(seq_length, height, width, channels))
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128, return_sequences=True)(flattened_features)
    lstm_out = BatchNormalization()(lstm_out)
    lstm_out = Dropout(0.4)(lstm_out)
    lstm_out = LSTM(128)(lstm_out)
    lstm_out = BatchNormalization()(lstm_out)
    lstm_out = Dropout(0.3)(lstm_out)

    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.3)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Training Preparation
cnn_lstm_model = build_cnn_lstm_model(seq_length=15, height=224, width=224, channels=3, num_classes=2)
cnn_lstm_model.summary()

indices = np.arange(1612)
train_indices, val_indices = train_test_split(indices, test_size=0.3, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_1612.dat', 
                          r'E:\PosePerfect\Dataset Creation\y_final_1612.dat', 
                          train_indices, batch_size=12)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_1612.dat', 
                        r'E:\PosePerfect\Dataset Creation\y_final_1612.dat', 
                        val_indices, batch_size=12)

checkpoint_dir = './mobilenetV2_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    return lr * 0.9 if epoch >= 2 else lr

lr_callback = LearningRateScheduler(scheduler)
step_timer_callback = StepTimerCallback()
batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

# Model Training
cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, step_timer_callback, batch_early_stopping_callback],
    verbose=1
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 15, 7, 7, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 15, 62720)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 15, 128)        │    32,178,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 15, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,577,666 (131.90 MB)

 Trainable params: 32,319,170 (123.29 MB)

 Non-trainable params: 2,258,496 (8.62 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/10
Step 1/94 - Loss: 1.3009, Accuracy: 0.5000, Time: 147.57 seconds
 1/94 ━━━━━━━━━━━━━━━━━━━━ 3:53:01 150s/step - accuracy: 0.5000 - loss: 1.3009Step 2/94 - Loss: 1.1387, Accuracy: 0.4583, Time: 5.11 seconds
 2/94 ━━━━━━━━━━━━━━━━━━━━ 7:59 5s/step - accuracy: 0.4792 - loss: 1.2198     Step 3/94 - Loss: 1.0540, Accuracy: 0.4444, Time: 3.83 seconds
 3/94 ━━━━━━━━━━━━━━━━━━━━ 6:51 5s/step - accuracy: 0.4676 - loss: 1.1645Step 4/94 - Loss: 1.0759, Accuracy: 0.4792, Time: 3.93 seconds
 4/94 ━━━━━━━━━━━━━━━━━━━━ 6:29 4s/step - accuracy: 0.4705 - loss: 1.1424Step 5/94 - Loss: 1.1272, Accuracy: 0.5000, Time: 3.92 seconds
 5/94 ━━━━━━━━━━━━━━━━━━━━ 6:16 4s/step - accuracy: 0.4764 - loss: 1.1393Step 6/94 - Loss: 1.0385, Accuracy: 0.5417, Time: 3.89 seconds
 6/94 ━━━━━━━━━━━━━━━━━━━━ 6:06 4s/step - accuracy: 0.4873 - loss: 1.1225Step 7/94 - Loss: 1.0174, Accuracy: 0.5357, Time: 3.92 seconds
 7/94 ━━━━━━━━━━━━━━━━━━━━ 5:58 4s/step - accuracy: 0.4942 - loss: 1.107